In [3]:
# import openpyxl module 
import openpyxl 
  
# Give the location of the file 
path = "/Users/mokrzesik/Desktop/Michael/General/Disney/05-01-2024 Disney Dining.xlsx"

# set the sheet names of the workbook
sheet1 = 'Restaurant List'
sheet2 = 'Menus'
  
# To open the workbook 
# workbook object is created 
wb = openpyxl.load_workbook(path) 
  
# Get menu workbook
# from the active attribute 
menu = wb.worksheets[1]

In [6]:
print(menu['B2'].value)

10.49


In [8]:
menu['L1'] = None
wb.save(path)